## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSSBBBB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
503
+------------------+-------------------+-------------------+-------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+--------------------+-------------------+------------------+-----------+-----------+------+--------------------+------+
|           coeff_1|            coeff_2|            coeff_3|            coeff_4|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|               res_3|              res_4|          res_mean|    station|  total_var|undefs|              vector|  year|
+------------------+-------------------+-------------------+-------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+--------------------+-------------------+------------------+-----------+-----------+------+--------------------+------+
|-4673.181412303238|-2309.3414111954517| 143.79242052207

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'coeff_4', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_4', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [6]:
#extract longitude and latitude for each station
feature='coeff_4'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_4'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_4 FROM weather


,station,year,coeff_4
0,USC00299820,1984.0,-465.003259
1,USC00299820,1985.0,316.578443
2,USC00298011,1968.0,-643.066868
3,USC00299820,1979.0,376.628768
4,USC00298011,1960.0,225.221780


In [7]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_4')
year_station_table.head(10)

station,US1NMBR0034,USC00290225,USC00290743,USC00290903,USC00291080,USC00291086,USC00291647,USC00291982,USC00292100,USC00292139,...,USC00298830,USC00298834,USC00299031,USC00299113,USC00299193,USC00299496,USC00299820,USW00023049,USW00023054,USW00093057
year,,,,,,,,,,,,,,,,,,,,,
1948.0,NaN,NaN,231.673776,NaN,NaN,279.017026,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,524.578086,NaN,NaN,NaN,NaN,448.181432,NaN
1949.0,NaN,NaN,71.889227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.326648,-44.221501,NaN
1950.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1951.0,NaN,NaN,89.857734,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1952.0,NaN,NaN,-8.170526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.032185,NaN
1953.0,NaN,NaN,NaN,NaN,NaN,NaN,95.883711,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-282.096874,NaN,77.615922,NaN
1954.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.555267,NaN
1955.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.390678,...,NaN,NaN,NaN,NaN,NaN,NaN,264.697638,NaN,NaN,NaN
1956.0,NaN,NaN,51.143334,NaN,123.011922,NaN,NaN,NaN,NaN,289.983529,...,121.898169,NaN,NaN,NaN,63.084576,NaN,1073.687953,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [8]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  380.574463507
RMS removing mean-by-station=  367.541407743
RMS removing mean-by-year   =  286.791188233


In [9]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 380.574463507
0 after removing mean by year    = 286.791188233
0 after removing mean by stations= 274.679594942
1 after removing mean by year    = 273.446877618
1 after removing mean by stations= 273.260002687
2 after removing mean by year    = 273.225887348
2 after removing mean by stations= 273.218708292
3 after removing mean by year    = 273.216989811
3 after removing mean by stations= 273.216519937
4 after removing mean by year    = 273.216371267
4 after removing mean by stations= 273.21631653
